In [1]:
import os
import time
import numpy as np
import pandas as pd
import cv2
from PIL import Image

import plotly.express as px
import plotly.graph_objects as go

import torch
from torchvision import transforms

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce GTX 1660 Ti


In [2]:
REAL_ESRGAN_INPUTS = "Real-ESRGAN/inputs/"
REAL_ESRGAN_OUTPUTS = "Real-ESRGAN/results/"

Parent_Data_Path = "Data 01/"

PAGES_PATH = "Data 01/Pages/"
PAGES_ENHANCED_PATH = "Data 01/Pages - Enhanced/"

WORDS_PATH = "Data 01/Words/"
WORDS_ENHANCED_PATH = "Data 01/Words - Enhanced/"

In [38]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5-word/runs/train/exp28/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\ahbaz/.cache\torch\hub\master.zip
YOLOv5  2022-3-3 torch 1.8.2+cu102 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)

Fusing layers... 
Model Summary: 213 layers, 7053277 parameters, 0 gradients
Adding AutoShape... 


In [3]:
def delete_RESRNGAN_inputs():
    for file in os.scandir(REAL_ESRGAN_INPUTS):
        os.remove(file.path)

In [4]:
def delete_RESRNGAN_outputs():
    for file in os.scandir(REAL_ESRGAN_OUTPUTS):
        os.remove(file.path)

# Page Enhancement

In [5]:
def Page_Clipper(img, Cutter_Square_Size = 100):
    Cutter_Page = []
    for rect in range(1, int(img.shape[0]/Cutter_Square_Size) + 2):
        r = img[(rect - 1) * Cutter_Square_Size: rect * Cutter_Square_Size,:]
        temp = []
        for sq in range(1, int(r.shape[1]/Cutter_Square_Size) + 2):
            s = r[:, (sq - 1) * Cutter_Square_Size: sq * Cutter_Square_Size]
            temp.append(s)
        Cutter_Page.append(temp)
    return Cutter_Page

In [6]:
delete_RESRNGAN_inputs()
delete_RESRNGAN_outputs()

In [7]:
def Page_Enhancement(page, Cutter_Square_Size = 100, factor = 0.5):
    Cutter_Page = Page_Clipper(page, Cutter_Square_Size)
    
    temp_final = []
    for r1 in Cutter_Page:
        delete_RESRNGAN_inputs()
        delete_RESRNGAN_outputs()
        i = 0
        for r2 in r1:
            cv2.imwrite(REAL_ESRGAN_INPUTS + "img " + chr(65 + i) + '.jpg', r2)
            i += 1

        time.sleep(1)
        torch.cuda.empty_cache()
        os.chdir('Real-ESRGAN/')
        !python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input inputs
        time.sleep(1)
        os.chdir('..')

        temp = []
        for r in os.scandir(REAL_ESRGAN_OUTPUTS):
            if r.is_file():
                img = Image.open(r.path)
                transform = transforms.Resize(size=(int(img.size[1] * factor), int(img.size[0] * factor)))
                img = np.asarray(transform(img))
                temp.append(img)
        rect = np.concatenate(temp, axis = 1)
        temp_final.append(rect)
        delete_RESRNGAN_inputs()
        delete_RESRNGAN_outputs()
    temp_final = np.concatenate(temp_final, axis = 0)
    return temp_final

In [8]:
i = 0
for page_path in os.scandir(PAGES_PATH):
    if page_path.is_file():
        page = Page_Enhancement(cv2.imread(page_path.path))
        cv2.imwrite(PAGES_ENHANCED_PATH + 'Page ' + chr(65 + i) + '.jpg', page)
        i += 1

Testing 0 img A
Testing 1 img B
Testing 2 img C
Testing 3 img D
Testing 4 img E
Testing 5 img F
Testing 6 img G
Testing 7 img H
Testing 8 img I
Testing 9 img J
Testing 0 img A
Testing 1 img B
Testing 2 img C
Testing 3 img D
Testing 4 img E
Testing 5 img F
Testing 6 img G
Testing 7 img H
Testing 8 img I
Testing 9 img J
Testing 0 img A
Testing 1 img B
Testing 2 img C
Testing 3 img D
Testing 4 img E
Testing 5 img F
Testing 6 img G
Testing 7 img H
Testing 8 img I
Testing 9 img J
Testing 0 img A
Testing 1 img B
Testing 2 img C
Testing 3 img D
Testing 4 img E
Testing 5 img F
Testing 6 img G
Testing 7 img H
Testing 8 img I
Testing 9 img J
Testing 0 img A
Testing 1 img B
Testing 2 img C
Testing 3 img D
Testing 4 img E
Testing 5 img F
Testing 6 img G
Testing 7 img H
Testing 8 img I
Testing 9 img J
Testing 0 img A
Testing 1 img B
Testing 2 img C
Testing 3 img D
Testing 4 img E
Testing 5 img F
Testing 6 img G
Testing 7 img H
Testing 8 img I
Testing 9 img J
Testing 0 img A
Testing 1 img B
Testing 

# Word Cuttouts

In [45]:
def Word_Extraction(page, min_conf = 0.75):
    page_result = model(page)
    
    coordinates = []
    words = []
    for e in page_result.crop():
        if e['conf'].item() >= min_conf:
            x1 = e['box'][0].item()
            y1 = e['box'][1].item()
            x2 = e['box'][2].item()
            y2 = e['box'][3].item()
            w = x2 - x1
            h = y2 - y1
            xm = x1 + (w/2)
            ym = y2 + (h/2)
            x1_1 = x1 + w
            y1_1 = y1
            coordinates.append([x1, y1, x2, y2, h, w, x1_1, y1_1, xm, ym])
            words.append(e['im'])

    df = pd.DataFrame(coordinates, columns=['x1', 'y1', 'x2', 'y2', 'h', 'w', 'x1_1', 'y1_1', 'xm', 'ym'])

    sorted_lines_index = []
    x_origin, y_origin = int(page.shape[1] / 2), 0

    while True:
        df['R'] = (abs(x_origin - df['x1']) + abs(y_origin - df['y1'])).to_list()
        index_ref = df.sort_values(by = ['R']).index[0]
        y_ref = df.loc[index_ref].y1
        h_ref = df.loc[index_ref].h
        print(index_ref)
        df['dis'] = (df.y1 - y_ref - (h_ref / 2)).to_list()
        # (df['dis'] < 0).sum()
        sorted_line_index = df[df['dis'] < 0].sort_values(by = ['x1']).index.to_list()
        sorted_lines_index.append(sorted_line_index)
        df.drop(sorted_line_index, inplace=True)
        del df['R']
        del df['dis']

        if df.shape[0] <= 0:
            break

    return sorted_lines_index, coordinates

In [46]:
for page_enhanced_path in os.scandir(PAGES_ENHANCED_PATH):
    if page_enhanced_path.is_file():
        page_name = page_enhanced_path.path.split('/')[-1].split('.')[0]
        page = cv2.imread(page_enhanced_path.path)
        sorted_lines_index, coordinates = Word_Extraction(page)
        
        if not os.path.isdir(WORDS_PATH + page_name):
            os.mkdir(WORDS_PATH + page_name, mode=511)
            
        i = 1
        for line in sorted_lines_index:
            for word in line:
                croped_word = page[int(coordinates[word][1]) : int(coordinates[word][3]), int(coordinates[word][0]) : int(coordinates[word][2])]

                cv2.imwrite(WORDS_PATH + page_name + '/' + 'Word ' + str(i) + '.jpg', croped_word)
                i += 1

Saved 1 image to runs\detect\exp3
Saved results to runs\detect\exp3



98
81
43
53
13
127
68
7
86
24
80
124
63
103


# Word Enhancement

In [11]:
def Word_Enhancement(words, factor = 0.5):
    delete_RESRNGAN_inputs()
    delete_RESRNGAN_outputs()

    i = 0
    for word in words:
        cv2.imwrite(REAL_ESRGAN_INPUTS + "img " + str(i) + '.jpg', word)
        i += 1

    time.sleep(1)
    torch.cuda.empty_cache()
    os.chdir('Real-ESRGAN/')
    !python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input inputs
    time.sleep(1)
    os.chdir('..')

    words_final = []
    for r in os.scandir(REAL_ESRGAN_OUTPUTS):
        if r.is_file():
            img = Image.open(r.path)
            transform = transforms.Resize(size=(int(img.size[1] * factor), int(img.size[0] * factor)))
            img = np.asarray(transform(img))
            words_final.append(img)
            
    delete_RESRNGAN_inputs()
    delete_RESRNGAN_outputs()

    return words_final

In [24]:
for page_name_path in os.scandir(WORDS_PATH):
    words = []
    for word_path in os.scandir(page_name_path.path):
        word = cv2.imread(word_path.path)
        words.append(word)
    words = Word_Enhancement(words)
    
    page_name = page_name_path.path.split('/')[-1]
    if not os.path.isdir(WORDS_ENHANCED_PATH + page_name):
        os.mkdir(WORDS_ENHANCED_PATH + page_name, mode=511)

    i = 1
    for word in words:
        cv2.imwrite(WORDS_ENHANCED_PATH + page_name + '/' + 'Word ' + str(i) + '.jpg', word)
        i += 1

Testing 0 img 0
Testing 1 img 1
Testing 2 img 10
Testing 3 img 100
Testing 4 img 101
Testing 5 img 102
Testing 6 img 103
Testing 7 img 104
Testing 8 img 105
Testing 9 img 106
Testing 10 img 107
Testing 11 img 108
Testing 12 img 109
Testing 13 img 11
Testing 14 img 110
Testing 15 img 111
Testing 16 img 112
Testing 17 img 113
Testing 18 img 114
Testing 19 img 115
Testing 20 img 116
Testing 21 img 117
Testing 22 img 118
Testing 23 img 119
Testing 24 img 12
Testing 25 img 120
Testing 26 img 121
Testing 27 img 122
Testing 28 img 123
Testing 29 img 124
Testing 30 img 125
Testing 31 img 126
Testing 32 img 127
Testing 33 img 128
Testing 34 img 129
Testing 35 img 13
Testing 36 img 14
Testing 37 img 15
Testing 38 img 16
Testing 39 img 17
Testing 40 img 18
Testing 41 img 19
Testing 42 img 2
Testing 43 img 20
Testing 44 img 21
Testing 45 img 22
Testing 46 img 23
Testing 47 img 24
Testing 48 img 25
Testing 49 img 26
Testing 50 img 27
Testing 51 img 28
Testing 52 img 29
Testing 53 img 3
Testing 54 i